In [1]:
import pandas as pd
import random 
import json


from functions import join_stocks_crypto, generate_rand_portfolios
from functions_post_clustering import simulate_evaluate_portfolio_subset

/Users/kirillkovalchuk/Desktop/Projects/thesis_clustering/venv/lib/python3.13/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
#GET THE DATA IN
df_all_stocks = pd.read_csv('stocks_data_filled.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')

joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left') #mode - either do left with crypto and fill NA for stocks or do left on stocks and leave out some dates for cryptos
joined_df.index = pd.to_datetime(joined_df.index)

returns_all = joined_df.pct_change()


tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)


#Reassemble the portfolio jsons for minvar
min_var_portfolios = dict()
for i in range(1,1000):
    with open(f'min_variance_portfolio_jsons/my_dict{i}.json') as f:
        port = json.load(f)
        min_var_portfolios.update(port)

with open(f"full_optimized_min_variance.json", "w") as f:
    json.dump(min_var_portfolios, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: 'min_variance_portfolio_jsons/my_dict1.json'

SIMULATE AND EVALUATE

In [ ]:
simulations_results_dict_rand, subset_statistics_df_rand, _ = simulate_evaluate_portfolio_subset(random_portfolios, returns_all, n_sims=100, t=100, distribution_model='multivar_norm')

simulations_results_dict_minvar, subset_statistics_df_minvar, _ = simulate_evaluate_portfolio_subset(min_var_portfolios, returns_all, n_sims=100, t=100, distribution_model='multivar_norm')

                           statistic        p_value  normal
mean_cumulative_return    586.218745  5.060773e-128   False
mean_daily_return         586.218745  5.060773e-128   False
std_cumulative_return     785.192641  3.144738e-171   False
std_daily_return          777.133328  1.768650e-169   False
sharpe_daily               12.979482   1.518942e-03   False
sharpe_cumulative          13.615903   1.104954e-03   False
sharpe_annual              12.979482   1.518942e-03   False
sharpe_cumulative_annual   13.615903   1.104954e-03   False
VaR                       598.947112  8.715396e-131   False
CVaR                      469.722552  1.002406e-102   False


In [ ]:
from scipy.stats import f_oneway
from scipy.stats import kruskal


def difference_between_subsets(subset_stats_dfs:list, metrics=['sharpe_annual'], test='anova'):
    tests_results = dict()
    for metric in metrics:
        groups = [subset_df[metric] for subset_df in subset_stats_dfs]
    
        if test == 'anova':
            test_stat, test_p = f_oneway(groups)
        elif test == 'kruskal':
            test_stat, test_stat = kruskal(groups)

        tests_results[metric] = {'test_stat': test_stat, 'test_p': test_p}

    return tests_results

In [ ]:
difference_between_subsets([subset_statistics_df_rand, subset_statistics_df_minvar], metric=['sharpe_annual'], test='kruskal')